# Libraries

In [2]:
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
# from transformers import T5Tokenizer
from transformers import AutoTokenizer

# Reading Data

In [4]:
df_squad = pd.read_excel('Arabic squad/Arabic-SQuAD.xlsx')
df_arcd = pd.read_excel('ARCD/arcd-train.xlsx')
df_tydiqa = pd.read_csv('tydiqa/tydiqa-arabic.csv')
df_mlqa = pd.read_excel('MLQA/MLQA-dev-context-ar-question-ar.xlsx')

# Data Overview

In [5]:
df_squad.head()

,id,question,context,answers,c_id
0,570bce516b8089140040fa42,ما هو ASCII على أساس؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': 'الأبجدية الإنجليزية', 'answer_start...",0
1,570bce516b8089140040fa43,كم شخصيات محددة موجودة في كود ASCII؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '128 حرف ا محدد', 'answer_start': 58}]",0
2,570bce516b8089140040fa44,كم عدد أحرف التحكم غير الطباعة؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '33 حرف ا تحكم ا غير الطباعة', 'answ...",0
3,570bce516b8089140040fa45,كم شخصيات قابلة للطباعة؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '95 حرف ا قابلا للطباعة', 'answer_st...",0
4,570bce516b8089140040fa46,ما هو الفضاء المعروف أيضا باسم ماذا؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': 'رسم ا غير مرئي 223', 'answer_start'...",0


In [10]:
df_squad.shape

(48344, 5)

In [6]:
df_arcd.head()

,id,question,context,answers,c_id
0,969331847966,- من هو جمال أحمد حمزة خاشقجي؟,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,"[{'text': 'صحفي وإعلامي', 'answer_start': 73}]",0
1,115150665555,- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,[{'text': 'حمزة خاشقجي (13 أكتوبر 1958، المدين...,0
2,74212080718,- في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,"[{'text': 'المدينة المنورة', 'answer_start': 39}]",0
3,465699296586,- في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,"[{'text': 'واشنطن بوست', 'answer_start': 224}]",1
4,564177542570,- كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,[{'text': 'وُصف في الصحف وأجهزة الاعلام العالم...,1


In [11]:
df_arcd.shape

(693, 5)

In [7]:
df_tydiqa.head()

,id,question,context,answers,c_id
0,arabic-2387335860751143628-1,كم عدد مرات فوز الأوروغواي ببطولة كاس العالم ل...,أقيمت البطولة 21 مرة، شارك في النهائيات 78 دول...,"[{'text': 'بطولتين', 'answer_start': 394}]",0
1,arabic--3358420169913421088-0,من هو مكتشف المرو أو الكوارتز ؟,المرو أو الكوارتز هو معدن يعود اكتشافه إلى ال...,"[{'text': '(بيير كوري) وأخوه (جاك)', 'answer_s...",1
2,arabic-6869798435672288559-0,كيف يتصل الجنين بالرحم ؟,المَشِيمَة أو الخَلاَص أو السُخْد (بالإنجليزي...,"[{'text': 'عن طريق الحبل السري', 'answer_start...",2
3,arabic-1608369850259830544-0,ما هي المَشِيمَة أو الخَلاَص أو السُخْد؟,المَشِيمَة أو الخَلاَص أو السُخْد (بالإنجليزي...,[{'text': 'عضو دائري مسطح الشكل يتصل بالجنين ع...,2
4,arabic--7165672200361826550-0,أين يقع مسجد السلطان عبد المجيد؟,مسجد السلطان عبد المجيد، هو مسجد أثري تاريخي ف...,"[{'text': 'مدينة جبيل اللبنانية', 'answer_star...",3


In [12]:
df_tydiqa.shape

(921, 5)

In [8]:
df_mlqa.head()

,id,question,context,answers,c_id
0,569666f4dc3983dab5624e989212c1d9d0cd1798,هل تزول الإصابة بمرض ذبابة الرمال عبر الوقت؟,تنتقل حمى الفواصد عن طريق لدغات حشرات من جنس ا...,"[{'text': 'وتبقى مصابة بالفيروس مدى حياتها', '...",0
1,561971f7978f678c3d1ba2a946036cdc131c4d49,ما الذي يتكون بالكامل تقريبا من خلايا نسيجية؟,تعد خلايا متن النبات خلايا حية لديها وظائف متن...,"[{'text': 'الأوراق', 'answer_start': 194}]",1
2,60ee75c50c8472be7cce1a24ee2cd7409ee4dd52,ما اسم نوع الخيوط الذي يحتوي على أجزاء من الكرفس؟,خلايا النسيج الغروي - تعيش خلايا النسيج الغروي...,"[{'text': 'النسيج الغروي.', 'answer_start': 11...",2
3,6a636c91f7733165c92ed84864debf6bdbdf7d16,ما هو ناتج تميز الخلايا النباتية؟,تختلف الطبقات الرئيسية من الخلايا عن الخلايا ا...,[{'text': 'هياكل أنسجة الجذور والسيقان والأورا...,3
4,18a5c05701b7359fcd32a379b2ac4a9a5d7544de,أي نوع من النباتات يحتوي على النسيج الوعائي ال...,تكون خلايا النسيج الوعائي الخشبي خلايا مطولةً...,"[{'text': 'تراشيوفيت', 'answer_start': 320}]",4


In [13]:
df_mlqa.shape

(517, 5)

In [9]:
df_squad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48344 entries, 0 to 48343
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        48344 non-null  object
 1   question  48344 non-null  object
 2   context   48344 non-null  object
 3   answers   48344 non-null  object
 4   c_id      48344 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


In [14]:
df_arcd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        693 non-null    int64 
 1   question  693 non-null    object
 2   context   693 non-null    object
 3   answers   693 non-null    object
 4   c_id      693 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 27.2+ KB


In [15]:
df_tydiqa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        921 non-null    object
 1   question  921 non-null    object
 2   context   921 non-null    object
 3   answers   921 non-null    object
 4   c_id      921 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 36.1+ KB


In [16]:
df_mlqa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        517 non-null    object
 1   question  517 non-null    object
 2   context   517 non-null    object
 3   answers   517 non-null    object
 4   c_id      517 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 20.3+ KB


# Removing id column

In [17]:
# TODO: Removing ids column in all datasets

In [18]:
df_squad.drop('id', axis=1, inplace=True)

In [19]:
df_squad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48344 entries, 0 to 48343
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  48344 non-null  object
 1   context   48344 non-null  object
 2   answers   48344 non-null  object
 3   c_id      48344 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [20]:
df_squad.head(10)

,question,context,answers,c_id
0,ما هو ASCII على أساس؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': 'الأبجدية الإنجليزية', 'answer_start...",0
1,كم شخصيات محددة موجودة في كود ASCII؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '128 حرف ا محدد', 'answer_start': 58}]",0
2,كم عدد أحرف التحكم غير الطباعة؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '33 حرف ا تحكم ا غير الطباعة', 'answ...",0
3,كم شخصيات قابلة للطباعة؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '95 حرف ا قابلا للطباعة', 'answer_st...",0
4,ما هو الفضاء المعروف أيضا باسم ماذا؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': 'رسم ا غير مرئي 223', 'answer_start'...",0
5,لماذا تم ترميز الرمز بحيث تكون معظم الرموز معًا؟,تم تزيين الرمز نفسه بحيث تكون معظم رموز التحكم...,"[{'text': 'لسهولة تحديد الهوية', 'answer_start...",1
6,كم عدد المراكز في العمودين الأولين؟,تم تزيين الرمز نفسه بحيث تكون معظم رموز التحكم...,"[{'text': '32 موضع', 'answer_start': 128}]",1
7,ما الذي قررت اللجنة أنه مهم؟,تم تزيين الرمز نفسه بحيث تكون معظم رموز التحكم...,[{'text': 'دعم الأحرف الهجائية الكبيرة المكونة...,1
8,أين كان الحرف ألف يضع في الموقف؟,تم تزيين الرمز نفسه بحيث تكون معظم رموز التحكم...,"[{'text': '41hex', 'answer_start': 757}]",1
9,تم دمج ASCII في ما مجموعة الأحرف الأخرى؟,تم دمج ASCII في مجموعة أحرف ونيكودي كأول 128 ر...,"[{'text': 'ونيكودي', 'answer_start': 28}]",2


# try1

tried to make something but found out it is the same as the original data

In [21]:
# # Assign a unique question_id to each question
# df_squad['question_id'] = df_squad.index

# # Function to get the context and its associated question_ids
# def get_context_question_ids(df):
#     grouped = df.groupby('c_id').agg(
#         context=('context', 'first'),
#         question_ids=('question_id', list)
#     ).reset_index()
#     return grouped

# # Generate the new DataFrame
# C_Q_df = get_context_question_ids(df_squad)

In [22]:
# C_Q_df.head(15)

In [23]:
# # Create a function to get the assigned questions
# def get_assigned_questions(df, c_id):
#     filtered = df[df['c_id'] == c_id]
#     Q = filtered['question'].tolist()
#     A = filtered['answers'].tolist()
#     return Q, A


In [24]:
# training_data = []

# for _, row in C_Q_df.iterrows():
#     context = row['context']
#     question_ids = row['question_ids']
    
#     # For each question_id in the context, fetch the corresponding question
#     for question_id in question_ids:
#         question = df_squad.loc[df_squad['question_id'] == question_id, 'question'].values[0]
#         training_data.append({
#             'context': context,
#             'question': question,
#             'answers': df_squad.loc[df_squad['question_id'] == question_id, 'answers'].values[0]
#         })
        
# training_df = pd.DataFrame(training_data)

In [25]:
# training_df.head(15)

In [26]:
# TODO: Arrange all data frames by their c_id column, to make same context after each other

In [27]:
# TODO: Combine all datasets as they already contain the same columns (pd.concat([df1, df2], ignore_index=True))

# Text Preprocessing

## text cleaning functions

### Normalization

In [28]:
def remove_diacritics(text):
    # remove diacritics from the text as it may confuse the model
    return re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)


### Text cleaning

In [29]:
def remove_non_arabic(text):
    # Remove non-Arabic characters, keeping essential punctuation
    text = re.sub(r'[^\u0600-\u06FF\s؟]', '', text)
    return text

In [30]:
def remove_punk(text):
    # Remove punctuation
    arabic_punctuation = r'[،؛؟…!"#$%&\'()*+,-./:;<=>@^_`{|}~]'
    text = re.sub(arabic_punctuation, '', text)
    return text

In [31]:
def remove_extra_spaces(text):
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [32]:
# def remove_alf(text):
#     alf = 'ا'
#     text = re.sub(alf, '', text)
#     return text

### Stop word removal

In [33]:
stop_words = set(stopwords.words('arabic'))
print(stop_words)

{'خاء', 'هَيْهات', 'كيفما', 'كأي', 'بَسْ', 'ثمَّ', 'أربعمائة', 'هيهات', 'ستمائة', 'أنشأ', 'حرى', 'حبيب', 'ي', 'علًّ', 'جمعة', 'ذين', 'لا سيما', 'حزيران', 'نبَّا', 'إذا', 'فإذا', 'إلَيْكَ', 'لكما', 'تسع', 'ريال', 'حيثما', 'ثمان', 'أوت', 'وَيْ', 'ثلاث', 'بكن', 'تخذ', 'مرّة', 'من', 'ر', 'بك', 'تلكم', 'ألف', 'اللاتي', 'أيّان', 'بلى', 'بكم', 'ثمانمئة', 'تبدّل', 'به', 'أولالك', 'لدن', 'هو', 'هاء', 'قام', 'وهب', 'أغسطس', 'آنفا', 'هاتان', 'هلا', 'أقل', 'إذ', 'سابع', 'ث', 'هَذَيْنِ', 'وإذ', 'صبرا', 'لبيك', 'اللتين', 'آناء', 'فإن', 'ذواتي', 'تعسا', 'ظلّ', 'مارس', 'فمن', 'مافتئ', 'شباط', 'فرادى', 'خلف', 'عشر', 'وراءَك', 'خال', 'ذلكما', 'آمينَ', 'تسعمائة', 'أربعمئة', 'قاف', 'ته', 'ذواتا', 'ولا', 'ع', 'أنا', 'أى', 'دينار', 'يوليو', 'قطّ', 'في', 'ضحوة', 'إي', 'هللة', 'مساء', 'خبَّر', 'قبل', 'لما', 'ليرة', 'أقبل', 'ذوا', 'أنتم', 'عين', 'كأن', 'بيد', 'تموز', 'ذَيْنِ', 'ثلاثمئة', 'حاشا', 'ّأيّان', 'أجمع', 'ماذا', 'أل', 'الذي', 'رأى', 'أنّى', 'أعلم', 'جميع', 'نعم', 'غالبا', 'ذه', 'والذين', 'جعل', 'إيهٍ'

In [34]:
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

### Handling numbers

In [35]:
def handle_numbers(text, replace_with=None):
    if replace_with:
        text = re.sub(r'\d+', replace_with, text)
    else:
        text = re.sub(r'\d+', '', text)
    return text

### Sentence splitting

In [36]:
def split_sentences(text):
    sentences = re.split(r'[؟.!؟]', text)
    # Remove empty sentences and strip spaces
    return [sentence.strip() for sentence in sentences if sentence.strip()]

### Tokenization

In [37]:
model_name = "aubmindlab/bert-base-arabertv2"   # replace the model with another when trying

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [38]:
# # Example Arabic text
# text = "مرحبا بك في عالم معالجة اللغة الطبيعية"

# # Tokenize the text
# encoded = tokenizer(text, return_tensors="pt")
# print("text after tokenize", encoded)

In [39]:
def t5_tokenize(text):
    return tokenizer(text, return_tensors="pt") # pt -> return tensors for pytorch

## Apply Functions on the text

In [40]:
df_squad.head(10)

,question,context,answers,c_id
0,ما هو ASCII على أساس؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': 'الأبجدية الإنجليزية', 'answer_start...",0
1,كم شخصيات محددة موجودة في كود ASCII؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '128 حرف ا محدد', 'answer_start': 58}]",0
2,كم عدد أحرف التحكم غير الطباعة؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '33 حرف ا تحكم ا غير الطباعة', 'answ...",0
3,كم شخصيات قابلة للطباعة؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': '95 حرف ا قابلا للطباعة', 'answer_st...",0
4,ما هو الفضاء المعروف أيضا باسم ماذا؟,يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، و...,"[{'text': 'رسم ا غير مرئي 223', 'answer_start'...",0
5,لماذا تم ترميز الرمز بحيث تكون معظم الرموز معًا؟,تم تزيين الرمز نفسه بحيث تكون معظم رموز التحكم...,"[{'text': 'لسهولة تحديد الهوية', 'answer_start...",1
6,كم عدد المراكز في العمودين الأولين؟,تم تزيين الرمز نفسه بحيث تكون معظم رموز التحكم...,"[{'text': '32 موضع', 'answer_start': 128}]",1
7,ما الذي قررت اللجنة أنه مهم؟,تم تزيين الرمز نفسه بحيث تكون معظم رموز التحكم...,[{'text': 'دعم الأحرف الهجائية الكبيرة المكونة...,1
8,أين كان الحرف ألف يضع في الموقف؟,تم تزيين الرمز نفسه بحيث تكون معظم رموز التحكم...,"[{'text': '41hex', 'answer_start': 757}]",1
9,تم دمج ASCII في ما مجموعة الأحرف الأخرى؟,تم دمج ASCII في مجموعة أحرف ونيكودي كأول 128 ر...,"[{'text': 'ونيكودي', 'answer_start': 28}]",2


In [41]:
# TODO: First see what preprocessing techniques we will use for the text for questions, contexts, answers(only text cleaning(i think))

# FIXME: Squad data is cleaned from the diacritics and been replaced with space, how to fix:
#           1- combine every ا to its previous word as the original word but some words may corrupt
#           2- combine every ى and أ with next word when coming in first
#           3- remove every alone character but this also maya corrupt some words as some words splitted from the diacritics (except for ب, ك and و)

In [42]:
# TODO: 2 approaches to be taken on the datasets:
#              1- apply the fixme changes to squad data
#              2- try the nlp preprocessing techniques to the other datasets (arcd, tydiqa, mlqa) and see if there any problems in them

### Questions column

In [43]:
df_squad['processed_question'] = df_squad['question'].apply(remove_diacritics).apply(remove_extra_spaces).apply(remove_punk)

* i tried to apply text clean function but i think it made some question not understandable

In [44]:
df_squad['processed_question'].head(10)

0                              ما هو ASCII على أساس
1               كم شخصيات محددة موجودة في كود ASCII
2                    كم عدد أحرف التحكم غير الطباعة
3                           كم شخصيات قابلة للطباعة
4               ما هو الفضاء المعروف أيضا باسم ماذا
5    لماذا تم ترميز الرمز بحيث تكون معظم الرموز معا
6                كم عدد المراكز في العمودين الأولين
7                       ما الذي قررت اللجنة أنه مهم
8                   أين كان الحرف ألف يضع في الموقف
9           تم دمج ASCII في ما مجموعة الأحرف الأخرى
Name: processed_question, dtype: object

In [45]:
list = []
for i in range(len(df_squad['question'])):
    c = df_squad['question'][i].count('؟')
    list.append(c)
    # print(df_squad['question'][0])
list

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


### Context column

In [50]:
df_squad['processed_context'] = df_squad['context'].apply(remove_punk)
df_squad['context'][df_squad['c_id'] == 22].tolist()
# df_squad['context'].head(1).tolist()

['خلق الغموض المتأصل للعديد من أحرف التحكم ، جنبا إلى جنب مع استخدامها التاريخي ، مشاكل عند نقل الملفات النص العادي بين الأنظمة . وأفضل مثال على ذلك هو مشكلة الخط الجديد في أنظمة التشغيل المختلفة . تطلبت أجهزة تيليتيبي إنهاء سطر النص بكل من كاررياغي ريتورن الذي يحرك رأس الطباعة إلى بداية السطر و ليني فيد الذي يعمل على دفع خط الورق بدون تحريك رأس الطباعة . يأتي اسم كاررياغي ريتورن من حقيقة أن الآلة الكاتبة التي تحمل الورقة قد تحركت على آلة كاتبة يدوية ، في حين بقي الوضع الذي ضربت فيه الشظايا الشريطية ثابت ا . اضطرت إلى دفع كامل العربة أرجعت إلى اليمين لوضع الهامش الأيسر للورقة للسطر التالي .',
 'خلق الغموض المتأصل للعديد من أحرف التحكم ، جنبا إلى جنب مع استخدامها التاريخي ، مشاكل عند نقل الملفات النص العادي بين الأنظمة . وأفضل مثال على ذلك هو مشكلة الخط الجديد في أنظمة التشغيل المختلفة . تطلبت أجهزة تيليتيبي إنهاء سطر النص بكل من كاررياغي ريتورن الذي يحرك رأس الطباعة إلى بداية السطر و ليني فيد الذي يعمل على دفع خط الورق بدون تحريك رأس الطباعة . يأتي اسم كاررياغي ريتورن من حقيقة أن الآلة

In [47]:
unique_contexts_df = pd.DataFrame(df_squad['context'].unique().tolist(), columns=["unique context"])
unique_contexts_df.to_csv("unique_context.csv", index=False, header=True)

### Answers column

In [48]:
df_squad.loc[0, 'question'] = df_squad['question'][0].replace('؟', '')
df_squad['question'][0]

'ما هو ASCII على أساس'

# Model

In [33]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch

# Load the pre-trained Arabic T5 model and tokenizer
model_name = "aubmindlab/aragpt2-large"  # Arabic T5 or a suitable Arabic model
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples['output'], padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = outputs['input_ids']
    return inputs

train_dataset = training_data.map(tokenize_function, batched=True)

# Define the Trainer arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

KeyboardInterrupt: 

In [22]:
from transformers import T5Tokenizer
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained("aubmindlab/aragpt2-large")

# Function to clean the text
def clean_text(text):
    # Remove unnecessary characters, HTML tags, or special symbols
    text = text.replace('\n', ' ').replace('\r', ' ')
    return text

# Apply cleaning to the context and questions
df['context'] = df['context'].apply(clean_text)
df['question'] = df['question'].apply(clean_text)

# Function to tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['context'], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples['question'], padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = outputs['input_ids']
    return inputs

# Apply tokenization
tokenized_data = df.apply(tokenize_function, axis=1)

# Convert to a suitable format for the Trainer

class QADataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}
        return item

train_dataset = QADataset(tokenized_data)

# Now you can use the train_dataset with the Trainer for training
# Define the Trainer arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
def generate_answer(context, question):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        output = model.generate(inputs['input_ids'])
        
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

# Example usage:
context = "This is context 1"
question = "What is the answer to Q1?"
answer = generate_answer(context, question)
print("Generated Answer:", answer)
